# Install requirements

In [ ]:
pip install -r drive/MyDrive/requirements.txt

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Prepare dataset for training the model

In [ ]:
import os
import shutil

import fitz
from PIL import Image
from imageai.Detection.Custom import CustomObjectDetection


def calc_percent(current, total, ndigits=2):
    return round(current / max(total, 1) * 100, ndigits)


def copy_label_images_data(src_dir='drive/MyDrive/pdf', dst_dir='drive/MyDrive/train_data/circle', train_data_ratio=0.8):
    """
    Copy the annotated pictures and marked files to the training data set folder
    :param src_dir: Annotated picture folder
    :param dst_dir: Training data folder
    :param train_data_ratio: Proportion of training data, 80% of the total data is used as training data by default
    """
    print('copy label images data: {} -> {}'.format(src_dir, dst_dir))

    # List of all picture folders
    pdf_dirs = []
    for dir_name in os.listdir(src_dir):
        if dir_name.startswith('.') or dir_name.endswith('.pdf'):
            continue

        dir_path = os.path.join(src_dir, dir_name)
        pdf_dirs.append(dir_path)

    # Training data set path
    train_data_dir = os.path.join(dst_dir, 'train')
    images_dir = os.path.join(train_data_dir, 'images')
    annotations_dir = os.path.join(train_data_dir, 'annotations')

    # Verify the data set path
    validation_dir = os.path.join(dst_dir, 'validation')
    images_valid_dir = os.path.join(validation_dir, 'images')
    annotations_valid_dir = os.path.join(validation_dir, 'annotations')

    # Find all the annotation data files
    annotations_files = []
    for pdf_dir in pdf_dirs:
        filenames = os.listdir(pdf_dir)
        filenames = list(filter(lambda name: name.endswith('.xml'), filenames))
        for filename in filenames:
            filename = filename.replace('.xml', '')
            filepath = pdf_dir
            annotations_files.append({
                'filename': filename,
                'path': filepath
            })

    # Calculate the number of training sets, and copy the data to the corresponding folders of the training set and the verification set
    train_data_cnt = round(len(annotations_files) * train_data_ratio)
    for item in annotations_files[:train_data_cnt]:
        filename = item['filename']
        filepath = item['path']

        dst_filename = '{}_{}'.format(os.path.basename(item['path']), filename)

        shutil.copyfile(
            os.path.join(filepath, '{}.jpg'.format(filename)),
            os.path.join(images_dir, '{}.jpg'.format(dst_filename))
        )

        shutil.copyfile(
            os.path.join(filepath, '{}.xml'.format(filename)),
            os.path.join(annotations_dir, '{}.xml'.format(dst_filename))
        )

    for item in annotations_files[train_data_cnt:]:
        filename = item['filename']
        filepath = item['path']

        dst_filename = '{}_{}'.format(os.path.basename(item['path']), filename)

        shutil.copyfile(
            os.path.join(filepath, '{}.jpg'.format(filename)),
            os.path.join(images_valid_dir, '{}.jpg'.format(dst_filename))
        )

        shutil.copyfile(
            os.path.join(filepath, '{}.xml'.format(filename)),
            os.path.join(annotations_valid_dir, '{}.xml'.format(dst_filename))
        )


def extract_images_from_pdf(pdf_dir):
    """
    Extract the pictures in the PDF file and store them in a folder named after the PDF file
    :param pdf_dir: The path of the folder where the PDF is stored
    """
    print('extract all pdf to images(jpg): {}'.format(pdf_dir))

    if not os.path.exists(pdf_dir):
        print('pdf dir not exists: {}'.format(pdf_dir))
        return

    # Get a list of all PDF file names
    pdf_names = os.listdir(pdf_dir)
    pdf_names = list(filter(lambda name: name.endswith('.pdf'), pdf_names))

    if not pdf_names:
        print('pdf file not found')
        return

    cnt = 1
    total = len(pdf_names)
    percent = calc_percent(cnt, total)
    print('{}% -> {}/{}'.format(percent, cnt, total))

    # Extract one by one
    for pdf_name in pdf_names:
        # PDF file name
        pdf_filename = os.path.join(pdf_dir, pdf_name)

        # Image output folder name
        img_output_dir = pdf_name.replace('.pdf', '')
        img_output_dir = os.path.join(pdf_dir, img_output_dir)

        # Delete the old picture output folder
        if os.path.exists(img_output_dir):
            shutil.rmtree(img_output_dir)

        # Create a new image output folder
        os.makedirs(img_output_dir)

        # Open PDF files
        with fitz.open(pdf_filename) as doc:
            page_cnt = 0
            page_total = doc.page_count

            # Traverse all pages
            for page in doc:
                img_filename = os.path.join(img_output_dir, 'page-{}.jpg'.format(page.number))

                # Get the bitmap of the page
                pix = page.get_pixmap()

                # Save the bitmap as a file
                img = Image.frombytes('RGB', [pix.width, pix.height], pix.samples)
                img.save(img_filename, 'PNG')

                page_cnt += 1
                page_percent = calc_percent(page_cnt, page_total)
                print('\rpdf: {}% -> {}/{}, page: {}% -> {}/{}'.format(percent, cnt, total, page_percent, page_cnt, page_total), end='')

        # Print the progress
        percent = calc_percent(cnt, total)
        cnt += 1
        print('{}% -> {}/{}'.format(percent, cnt, total))

In [ ]:
pdf_dir = 'drive/MyDrive/pdf'
extract_images_from_pdf(pdf_dir)

# Copy the training data set

In [ ]:
import os
import shutil

def copy_label_images_data(src_dir='drive/MyDrive/pdf', dst_dir='drive/MyDrive/train_data/circle', train_data_ratio=0.8):
    """
    Copy the annotated pictures and marked files to the training data set folder
    :param src_dir: Annotated picture folder
    :param dst_dir: Training data folder
    :param train_data_ratio: Proportion of training data, 80% of the total data is used as training data by default
    """
    print('copy label images data: {} -> {}'.format(src_dir, dst_dir))

    # List of all picture folders
    pdf_dirs = []
    for dir_name in os.listdir(src_dir):
        if dir_name.startswith('.') or dir_name.endswith('.pdf'):
            continue

        dir_path = os.path.join(src_dir, dir_name)
        pdf_dirs.append(dir_path)

    # Training data set path
    train_data_dir = os.path.join(dst_dir, 'train')
    images_dir = os.path.join(train_data_dir, 'images')
    annotations_dir = os.path.join(train_data_dir, 'annotations')

    # Verify the data set path
    validation_dir = os.path.join(dst_dir, 'validation')
    images_valid_dir = os.path.join(validation_dir, 'images')
    annotations_valid_dir = os.path.join(validation_dir, 'annotations')

    # Find all the annotation data files
    annotations_files = []
    for pdf_dir in pdf_dirs:
        filenames = os.listdir(pdf_dir)
        filenames = list(filter(lambda name: name.endswith('.xml'), filenames))
        for filename in filenames:
            filename = filename.replace('.xml', '')
            filepath = pdf_dir
            annotations_files.append({
                'filename': filename,
                'path': filepath
            })

    # Calculate the number of training sets, and copy the data to the corresponding folders of the training set and the verification set
    train_data_cnt = round(len(annotations_files) * train_data_ratio)
    for item in annotations_files[:train_data_cnt]:
        filename = item['filename']
        filepath = item['path']

        dst_filename = '{}_{}'.format(os.path.basename(item['path']), filename)

        shutil.copyfile(
            os.path.join(filepath, '{}.jpg'.format(filename)),
            os.path.join(images_dir, '{}.jpg'.format(dst_filename))
        )

        shutil.copyfile(
            os.path.join(filepath, '{}.xml'.format(filename)),
            os.path.join(annotations_dir, '{}.xml'.format(dst_filename))
        )

    for item in annotations_files[train_data_cnt:]:
        filename = item['filename']
        filepath = item['path']

        dst_filename = '{}_{}'.format(os.path.basename(item['path']), filename)

        shutil.copyfile(
            os.path.join(filepath, '{}.jpg'.format(filename)),
            os.path.join(images_valid_dir, '{}.jpg'.format(dst_filename))
        )

        shutil.copyfile(
            os.path.join(filepath, '{}.xml'.format(filename)),
            os.path.join(annotations_valid_dir, '{}.xml'.format(dst_filename))
        )

In [ ]:
copy_label_images_data()

# Annotate the data set

# Correct the annotated data

In [ ]:
import os
import re

def correct_train_data():
    xml_dirs = (
        os.path.join('drive','MyDrive','train_data', 'circle', 'train', 'annotations'),
        os.path.join('drive','MyDrive','train_data', 'circle', 'validation', 'annotations')
    )

    for xml_dir in xml_dirs:
        xml_file_paths = os.listdir(xml_dir)
        xml_file_paths = list(filter(lambda name: name.endswith('.xml'), xml_file_paths))

        for xml_file_path in xml_file_paths:
            print('correct train data: {}'.format(xml_file_path))
            img_filename = xml_file_path.replace('.xml', '.jpg')
            xml_file_path = os.path.join(xml_dir, xml_file_path)
            with open(xml_file_path, 'r+') as fp:
                xml_text = fp.read()
                xml_text = re.sub('<filename>.*</filename>', '<filename>{}</filename>'.format(img_filename), xml_text)
                fp.seek(0)
                fp.write(xml_text)

In [ ]:
correct_train_data()

# Start training the model

In [ ]:
import tensorflow as tf
from imageai.Detection.Custom import DetectionModelTrainer


def train():
    trainer = DetectionModelTrainer()
    trainer.setModelTypeAsYOLOv3()
    trainer.setDataDirectory(data_directory=os.path.join('drive','MyDrive','train_data', 'circle'))
    trainer.setTrainConfig(
        object_names_array=['circle'],
        batch_size=4,
        num_experiments=200,
        train_from_pretrained_model='pretrained-yolov3.h5'
    )
    trainer.trainModel()


def init_tf():
    # Set up GPU video memory to apply on demand to prevent insufficient video memory
    gpus = tf.config.experimental.list_physical_devices('GPU')
    if gpus:
        try:
            # Currently, memory growth needs to be the same across GPUs
            for gpu in gpus:
                tf.config.experimental.set_memory_growth(gpu, True)
            logical_gpus = tf.config.experimental.list_logical_devices('GPU')
            print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
        except RuntimeError as e:
            # Memory growth must be set before GPUs have been initialized
            print(e)

    print_is_gpu_available()


def print_is_gpu_available():
    print('gpu available: {}'.format(tf.test.is_gpu_available()))

In [ ]:
init_tf()
train()

# After training, test the recognition effect

In [ ]:
from imageai.Detection.Custom import CustomObjectDetection

def test(img_path, img_out, minimum_percentage_probability=10):
    """
    Recognition test
    :param img_path \\Image file path
    :param img_out \\Result image path
    :param minimum_percentage_probability \\Minimum similarity of graphics
    """
    print('test: {} -> {}'.format(img_path, img_out))

        '''
    /***************************************************************************************
*    Title: ImageAI : Custom Object Detection
*    Author: ImageAI Developers
*    Date: 2019
*    Code version: latest
*    Availability: https://github.com/OlafenwaMoses/ImageAI/blob/master/imageai/Detection/Custom/CUSTOMDETECTION.md
*
***************************************************************************************/'''

    # Load the model
    detector = CustomObjectDetection()
    detector.setModelTypeAsYOLOv3()
    detector.setModelPath('drive/MyDrive/train_data/circle/models/detection_model-ex-138--loss-0014.575.h5')
    detector.setJsonPath('drive/MyDrive/train_data/circle/models/detection_config.json')
    detector.loadModel()

    # Detect
    detections = detector.detectObjectsFromImage(
        input_image=img_path,
        output_image_path=img_out,
        minimum_percentage_probability=minimum_percentage_probability
    )

    # Print the result
    for detection in detections:
        print("{}: {} -> {}".format(detection['name'], detection['percentage_probability'], detection['box_points']))

In [ ]:
# Recognition effect test
img_test = 'test.jpg'
img_out = 'test_out.jpg'
test(img_test, img_out)